# Chapter 1 - Input to LLMs


<rewrite>
All generative LLMs are designed to take in some text and then predict what text is most likely to come after it.

"Base" models are trained on a wide variety of different texts, so they make minimal assumptions about the structure of the text they're completing.

Chat models are created from base models by training them on transcripts of dialogue, so they assume that whatever text they are given is a fragment of dialogue. You can chat with them by filling in one side of a conversation and letting the model fill in the other.

Instruct models are trained on instruction–response pairs, so if you give it an instruction, the model assumes that it should continue with a response that obeys the instruction.


https://langroid.github.io/langroid/blog/2023/09/19/language-models-completion-and-chat-completion/


## Introduction

LLM models take input as text and produce output as text. However, deep learning networks cannot work with text symbols. Hence the text
must be represented in a continuous space. In this chapter, we will look at how to pre-process text input so it's malleable for 
a large language model consumption.

Figure 1 shows the basic blocks of this operation.
(reference:preprocessing)=
```{figure} ../images/chapter1/input_creation.png
---
height: 150px
name: preprocessing
---
Input preprocessing blocks
```
In subsequent sections we will see each of those boxes in detail. Starting with token encoding, followed by word embedding and finally position embedding. In a nutshell this chapter is about how to prepare the data for ingestion into a large language model. Before we understand the nueances involved in each of these steps,
let us do a simple outline of what happens in each of these blocks using a no frill example.

(reference:nofrill)=
### No frill example

A causal large language model, also refered to autoregresive model is trained to do the next word prediction. From a vocabulary of words, given a sequence of words, the causal model predicts the most probable next word from the vocabulary. Causal model are trained on a bunch of documents. Documents are composed of words and each word is composed of characters. In our example, word will be lowest denomination of operation.We use the term corpus to refer to these input documents. The lowest denomination in this corpus is word, typically referred called tokens. The set of unique tokens in the corpus is called vocabulary.

Let us take a sample paragraph, our corpus for this exercise, and apply a regular expression to split the paragraph by whitespace or special characters. The resultant list of words forms the tokens for this corpus.

In [3]:
import re

# /* Text borrowed from https://www.tntech.edu/cas/physics/aboutphys/about-physics.php */
text_corpus = "Broadly, physics involves the study of everything in physical existence," + \
              "from the smallest subatomic particles to the entire universe. Physicists try " + \
              "to develop conceptual and mathematical models that describe interactions between entities " + \
              "(both big and small) and that can be used to extend our understanding of how the "+ \
               "universe works at different scales. Are you interested in studying physics?"


split_expr = r'([?.#$&*^@,)(]|\s)'
tokens = re.split(split_expr, text_corpus)
tokens = [token.strip() for token in tokens if len(token.strip()) > 0]
print(tokens)

['Broadly', ',', 'physics', 'involves', 'the', 'study', 'of', 'everything', 'in', 'physical', 'existence', ',', 'from', 'the', 'smallest', 'subatomic', 'particles', 'to', 'the', 'entire', 'universe', '.', 'Physicists', 'try', 'to', 'develop', 'conceptual', 'and', 'mathematical', 'models', 'that', 'describe', 'interactions', 'between', 'entities', '(', 'both', 'big', 'and', 'small', ')', 'and', 'that', 'can', 'be', 'used', 'to', 'extend', 'our', 'understanding', 'of', 'how', 'the', 'universe', 'works', 'at', 'different', 'scales', '.', 'Are', 'you', 'interested', 'in', 'studying', 'physics', '?']


:::{note}
In the above example we have a r before the string. This informs python interpreter to treat
blackslash as raw character and not as escape character.
:::

The set of unique tokens forms our vocabulary. Further we assign a unique id for each token.

In [4]:
vocabulary = {token:token_id for token_id, token in enumerate(set(tokens))}
print(vocabulary)

{'and': 0, 'interested': 1, 'to': 2, 'models': 3, 'small': 4, 'at': 5, 'scales': 6, 'universe': 7, ',': 8, 'entities': 9, 'subatomic': 10, 'that': 11, 'interactions': 12, 'be': 13, 'between': 14, 'particles': 15, 'our': 16, 'used': 17, 'can': 18, '.': 19, 'conceptual': 20, 'of': 21, 'involves': 22, 'everything': 23, 'entire': 24, 'smallest': 25, 'big': 26, 'understanding': 27, 'works': 28, 'mathematical': 29, 'studying': 30, 'Physicists': 31, 'the': 32, 'physical': 33, '(': 34, 'different': 35, 'study': 36, ')': 37, 'describe': 38, 'how': 39, 'try': 40, 'physics': 41, 'develop': 42, 'you': 43, 'existence': 44, '?': 45, 'Broadly': 46, 'from': 47, 'both': 48, 'in': 49, 'extend': 50, 'Are': 51}


With this vocabulary we can now encode any input string into a list of integers / token ids.

:::{note}
Size of the vocabulary plays a great part in building the LLM. The challenge is to have a very compact vocabulary and
still try to cover maxium amount of tokens in the corpus and also the token expectd in the future. We will discuss
more in this chapter about modeling exercise to build a compact vocabulary. The illustration given here is a very
simplified example.
:::

In [5]:
input_text = "universe works at different scales"

tokens = re.split(split_expr, input_text)
tokens = [token.strip() for token in tokens if len(token.strip()) > 0]
encoded_input = [vocabulary[token.strip()] for token in tokens]
print(encoded_input)

[7, 28, 5, 35, 6]


We have succesfully converted our word tokens into token id. Though this is now in number space, neural networks cannot process it. We need the input in a continous space. Here is where word embedding comes in handy. Let us build a embedding lookup table. The keys of this look up table are our integer word token ids. The value are a continous representation.


In [6]:
import numpy as np

vocab_size = max(vocabulary.values())
print(f"vocabulary size {vocab_size}")

embedding_size = 5
word_embedding = np.random.uniform(size=(vocab_size, embedding_size))
print(f"Word Embedding shape {word_embedding.shape}")
print(word_embedding[0:5,0:5])

vocabulary size 51
Word Embedding shape (51, 5)
[[0.79061896 0.23025439 0.55998829 0.8374295  0.40650912]
 [0.28080375 0.6848087  0.3396529  0.17248223 0.29623326]
 [0.07165954 0.76513322 0.47325477 0.7382186  0.76346754]
 [0.90756471 0.14178423 0.65096936 0.23639402 0.67450719]
 [0.37767153 0.42207631 0.68086117 0.59773189 0.78474626]]


Here we build an embedding lookup table. Our embedding dimension is set to 5. We create a look up table where rows represent the token and 
the columns represent the embedding for those words. The embeddings are random real numbers representing the words in a continous space.

In [7]:
input_embedding = word_embedding[encoded_input,:]
print(input_embedding.shape)
print(input_embedding[0:5, 0:5])

(5, 5)
[[0.85432497 0.33774429 0.5163072  0.16599111 0.77619878]
 [0.8367271  0.02581914 0.05084087 0.15658079 0.86089508]
 [0.25121756 0.13697472 0.44487938 0.13113087 0.51111093]
 [0.33662536 0.6617542  0.16918677 0.71635738 0.5714265 ]
 [0.51201298 0.95930028 0.74252266 0.45792642 0.87873272]]


Word positions carry semantic information. In addition to the words, providing the position of the words
will be benefial to the model. Similar to word embedding, we will create a look up for the position embedding.
Let us assume a simple case here. The input size to our LLM is fixed, say 10. We will call it as the sequence length.


In [8]:
sequence_length = 10
position_embedding_lookup = np.random.uniform(size=(sequence_length, embedding_size))

position_index =  np.arange(input_embedding.shape[0])
position_embedding = position_embedding_lookup[position_index, :]

position_embedding[0:2, 0:5]

array([[0.5235984 , 0.60912799, 0.66000757, 0.87457871, 0.50215997],
       [0.28703682, 0.38816409, 0.02410332, 0.85221138, 0.70862048]])

The embedding size is same as the word embedding. Finally we can now add the position embedding to word embedding



In [9]:
final_embedding = input_embedding + position_embedding

Typical of any deep learning model, feature values X and label value Y are fed into Large language model. The main job of a casual model is to predict the next given word. So say if we given "universe", based on how its trained it should be returning "works".


Let us see how we can quickly prepare the input X and the label Y for our LLM.


In [15]:
tokens = re.split(split_expr, text_corpus)
tokens = [token.strip() for token in tokens if len(token.strip()) > 0]
token_encoding = [vocabulary[token] for token in tokens]

feature_batch = []
label_batch = []

slide = 1
for idx in range(len(tokens) - sequence_length ) :
    feature = token_encoding[idx:idx + sequence_length]
    label =   token_encoding[idx + slide: idx + slide + sequence_length]

    feature_batch.append(feature)
    label_batch.append(label)

print(f"a feature : {feature_batch[0]}")
print(f"a label   : {label_batch[0]}")


a feature : [46, 8, 41, 22, 32, 36, 21, 23, 49, 33]
a label   : [8, 41, 22, 32, 36, 21, 23, 49, 33, 44]


Givent he token id 7, we want the LLM to predict 35, now given 35 we want it to predict 16 and so on. By sliding the feature 1 level to the right, we get the token ids for the labels. 


:::{note}
Sliding is a design decision. For demonstration purpose we have used a slide of 1. This may lead to overfitting in some cases.
:::

With these we can further get the embeddings throught he lookup table we have created.



Hopefully this gives a summary of all the steps involved in preparing the input for a LLM. Rest of the chapter will dwell into the details of token encoding and embeddings. Towards the end of the chapter we will introduce transformer and dataset python library from hugging face which provides convienient implementation of all the topics we discuss here.

In [ ]:
:::{note}
:::

## A Sample Text corpus

Publicly and privately available LLMs leverage the text data available in world wide web to do the pre-training. In the no frill section, we showed how the features and labels needed to train an LLM comes from the same source, sliding the features leaves us with the label. This can done in an unsupervised manner, saving the labor needed to create large training dataset. In the GPT-1 paper {cite}`radford2018improving`, the authors call this training process as unsupervised pre-training. GPT-1 was trained with Bookcorpus dataset {cite}`zhu2015aligning`.

Later in this book,  dwelling about RAG, we have shown a Q&A example with [Project Gutenberg](https://www.gutenberg.org/).


Loading input text from desparate sources is a tedious undertaking. LLMs are trained on Terra Bytes of data. Complex data pipelines are writeen to
extract and validate the data. Details of those pipelines are beyond the scope of the book. Here is a quote from {cite}`touvron2023llama`, "Our training corpus includes a new mix of data from publicly available sources, which does not include data
from Meta’s products or services. We made an effort to remove data from certain sites known to contain a
high volume of personal information about private individuals. We trained on 2 trillion tokens of data as this
provides a good performance–cost trade-off, up-sampling the most factual sources in an effort to increase
knowledge and dampen hallucinations."  

Read the section about safety in pre-training data to get some more details into the pre-processing works which goes on to accidently prevent sensitive information from leaking into training the LLM.

To give an idea about loading the corpus, we will use Simplebooks {cite}`nguyen2019simplebooks`. After downloading the dataset, we will show how to leverage hugginface's dataset libary to load the dataset, let us quickly explore the dataset.


In [335]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

def download_simplebooks():
    """
    Download simple books dataset
    """
    url = "https://dldata-public.s3.us-east-2.amazonaws.com/simplebooks.zip"
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path="../data/")
    print(f"Finished downloading and extracting simplebooks.zip")
                      
    
download_simplebooks()

Finished downloading and extracting simplebooks.zip


#### Structure of simplebooks

From project gutenberg, 1573 books were selected, mostly children book and simplebooks dataset was created.
Simplebooks, when downloaded comes with datasets in two sizes.Simplebooks-2 is of size 11MB with a vocabulary size of 11,492 and Simplebooks-92
of size roughly 400MB with a vocabulary size of 98,304. Simplebooks-2 has 2.2 M tokens. Compared to llama-2 which uses 2 trillion tokens, Simplebooks
is a small dataset which can be used write code to study LLMs.

    !ls ../data/simplebooks
    README.md  simplebooks-2  simplebooks-2-raw  simplebooks-92  simplebooks-92-raw

Both simplebooks-2 and simplebooks-92 has folders with raw suffix. The raw suffixed folders have the data with no changes from gutenberg source. The following normalization were performed on raw suffixed folders and the results are in non raw suffixed folders. 

1. Spacy was used to tokenize each book. Original case and punctuations were preserved.
2. @ was added as separator for numbers. So 300,000 becomes 300 @,@ 000.

Each of the folder have train, test and validation split and vocabulary files.

    !ls ../data/simplebooks/simplebooks-2
    test.txt  train.txt  train.vocab  valid.txt
    
simplebooks-2 and simplebooks-92 have the cleaned up data. The vocabulary built after applying pre-tokenization on the normalized text is also stored. A quick peek at the train files should show the difference.


In [23]:
!head -n 15 ../data/simplebooks/simplebooks-2/train.txt


More <unk> Tales

By

Ellen C. <unk>



I

The Girl Monkey And The <unk> Of <unk>


One day the king went for a long walk in the woods . When he came back to his own garden , he sent for his family to come down to the lake for a swim .



In [24]:
!head -n 15 ../data/simplebooks/simplebooks-2-raw/train.txt


More Jataka Tales

By

Ellen C. Babbitt



I

The Girl Monkey And The String Of Pearls


One day the king went for a long walk in the woods. When he came back to his own garden, he sent for his family to come down to the lake for a swim.



In [44]:
vocabulary = {}
with open("../data/simplebooks/simplebooks-2/train.vocab") as f:
    rows = ( line.split('\t') for line in f )
    count = 0
    for row in rows:
        vocabulary[row[0]] = int(row[1].strip())
        count+=1
        
print(f"Entries in vocabulary {count}")
print(f"sample tokens {list(vocabulary.keys())[0:5]}")
print(f"their encodings {list(vocabulary.values())[0:5]}")

Entries in vocabulary 11493
sample tokens [',', '.', 'the', '"', 'and']
their encodings [131695, 105703, 98932, 97156, 63612]


As a part of pre-tokenization some of the uknown words like Jataka, Babbitt, are replaced by a token ""<unk>"". More about special tokens later in this chapter. Unnecessary white space are removed, look at the sentence
"own garden , " is cleaned up to "own garden," Let us peek intot he vocabulary creaated.

Hopefully this gives an idea about input text corpus. We saw that Simplebooks used Spacy to clean up the text as a part of their normalization process and used a whitespace tokenization for thier pre-tokenization.

## Tokenization Pipeline

The tokenization begins with raw input text source / corpus and ends with a dictionary of tokens and their associated token ids. Token ids are integers. After this given a new text, the pipeline should be able to spit out the associated tokens. Similarly, given a list of tokens, the pipeline should be able to convert it back to text without any loss. The below figure illustrates the various steps involved in this pipeline.

(reference:tokenization)=
```{figure} ../images/chapter1/TokenEncoding.jpg
---
height: 250px
name: encoding
---
Steps in Tokenization
```

### Normalization

In the simplebooks example, we saw that unncessary whitespaces were removed and numbers were formatted by inserting '@' at different separators. 
Typicall normalization involves removing unncessary whitespaces, stripping of accents, lower case conversion and similar others. Here is a list of some normalizer s provided by [HuggingFace Tokenizer library](https://huggingface.co/docs/tokenizers/en/components).

1. Unicode normalization (NFD, NFKD, NFC and NFKC algorithms)
2. Lowecase conversion
3. Stripping white spaces and accents
4. Replacing common string patterns

:::{admonition} Unicode normalization

Unicode encoding involves assigning a numerical value called "code point" to each character and transforming them into a series of bytes.
Issues may arise when a character can be represented by a single code point or a combination of two code points. Unicode normalization
is the process of normalizing a unicode encoded string into a canonical form.

For the more curious please read the [article](https://www.smashingmagazine.com/2012/06/all-about-unicode-utf8-character-sets/) to get a little history about ASCII, latin-1, unicode.

:::

Quoting from GPT-1 paper {cite}`radford2018improving`, "We use the ftfy library2 to clean the raw text in BooksCorpus, standardize some punctuation and whitespace, and use the spaCy tokenizer".

1. [ftfy - fixes text for you](https://ftfy.readthedocs.io/en/latest/index.html)
2. [Spacy](https://spacy.io/)

Going into the details of ftfy and spacy is beyond the scope of this book. Following code snippets demonstrates the basic usage of these packages. We will discuss Spacy in pre-tokenization section.

In [20]:
import ftfy

ftfy.fix_text("L&AMP;AMP;ATILDE;&AMP;AMP;SUP3;PEZ")

'LóPEZ'

The string "L&AMP;AMP;ATILDE;&AMP;AMP;SUP3;PEZ" is converted to LoPEZ by ftfy. This package can take care of issues with character decoding. Let us look
at a spacy example. After installing spacy, download the tokenizer model to run the following code snipped.

    conda install ftfy spacy
    python -m spacy download en_core_web_sm

`````{admonition} Mojibake (文字化け, "Garbled") 
:class: tip
Garbled text formed as a result of being decoded using a character encoding with which it was not orignally encoded. 
A funny poem about Mojibake related to characters printed in a shippling label.

(reference:mojibake)=
Figure 2 A funny mojibake poem

```{figure} ../images/chapter1/shipping-label.png
---
height: 150px
name: shipping-label
---
Mojibake shipping label
```
ODE TO A SHIPPING LABEL
Once there was a little o,
with an accent on top like so

It started out as UTF8,
but the program only knew latin1,
and changed the litte o to A for fun.

and it goes on. For the complete [poem](https://imgur.com/4J7Il0m)

The text in the label is lopez and due to wrong decoding we have a Mojibake. 

`````

### Pre-tokenization


Using a set of rules, the text is split into atomic units, tokens. Imaging this as a superset of tokens fed into the vocabulary building exercise. A subset of these tokens make their way into the final vocabulary. An example pre-tokenizer is  a simple whitespace tokenizer. If two words are separated by a whitespace, they will be treated as two tokens.
We saw an example of this in the no frill section. Let us write some python code to implement what we have learnt.

In [2]:
import re
import ftfy
import spacy

class SpacyTokenizer():
    """
    Tokenizer based on Spacy library
    """
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")

    def __call__(self, input_text):

        assert len(input_text) > 0

        doc = nlp(input_text)
        tokens = [token.text for token in doc]
        tokens = [token.strip() for token in tokens if len(token.strip()) > 0]

        return tokens
        


class RegexTokenizer():
    """
    Regex Based Tokenizer
    Splits text by eitehr whitespace or by one of these
    special characters,?.#$&*^@,
    """
    def __call__(self, input_text):

        assert input_text is not None
        assert len(input_text) > 0

        tokenizer_regex = r'([?.#$&*^@,)(]|\s)'
        tokens = re.split(tokenizer_regex, input_text)
        tokens = [token.strip() for token in tokens if len(token.strip()) > 0]

        return tokens
        


/home/gopi/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025824022/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


The regex based tokenizer, uses the regex expression we introduced in no frills section. Spacy tokenizer uses
the Spacy library to tokenize. Let us take a sample from our Simplebooks dataset to see these tokenizers in action.

In [5]:
def read_simplebooks(path):
    for line in open(path, 'r'):
        yield line

simplebooks_reader = read_simplebooks('../data/simplebooks/simplebooks-2-raw/train.txt')
SAMPLE_SIZE = 15

tokenizer = RegexTokenizer()


simple_books_sample = [tokenizer(line) for idx, line in enumerate(simplebooks_reader) if idx <= SAMPLE_SIZE and len(line) > 1]
simple_books_sample

[['More', 'Jataka', 'Tales'],
 ['By'],
 ['Ellen', 'C', '.', 'Babbitt'],
 ['I'],
 ['The', 'Girl', 'Monkey', 'And', 'The', 'String', 'Of', 'Pearls'],
 ['One',
  'day',
  'the',
  'king',
  'went',
  'for',
  'a',
  'long',
  'walk',
  'in',
  'the',
  'woods',
  '.',
  'When',
  'he',
  'came',
  'back',
  'to',
  'his',
  'own',
  'garden',
  ',',
  'he',
  'sent',
  'for',
  'his',
  'family',
  'to',
  'come',
  'down',
  'to',
  'the',
  'lake',
  'for',
  'a',
  'swim',
  '.'],
 ['When',
  'they',
  'were',
  'all',
  'ready',
  'to',
  'go',
  'into',
  'the',
  'water',
  ',',
  'the',
  'queen',
  'and',
  'her',
  'ladies',
  'left',
  'their',
  'jewels',
  'in',
  'charge',
  'of',
  'the',
  'servants',
  ',',
  'and',
  'then',
  'went',
  'down',
  'into',
  'the',
  'lake',
  '.']]

With a sample of 15 sentences we are ready to pass it to our tokenizer.

### Tokenizer models - Dictionary Training

One may wonder the need for any subsequent processing in tokenization pipeline. The pre-tokenization output can be used directly to build a Vocabulary. The set of unique tokens gathered after running the tokenizer over the input corpus is the vocabulary. You are not alone. The transformerXL model {cite}`dai2019transformerxl` has a vocabulary size of 250K, compared to Llama which has a size of 32K

:::{note}
TransformerXL uses space and punctuation to tokenize the text. Their vocabulary size is around 250K. Here is the link
to their paper [Transformer-XL: Attentive Language Models Beyond a Fixed-Length Context](https://arxiv.org/abs/1901.02860)
::: 


A compact vocabulary reduces the model complexity and computation needs to train and perform inference. Special tokens and respective token-ids are added for unknown words. Say an input to the language model contains a word not present in the vocabulary, it will be treatd as unknown and the token id assigned for unknown word will be substituted. A good token encoding pipeline should strive to reduce the number of unknown words. Compact vocabulary and reduced unkown words are two opposite contraints.

Word based tokenization suffers from very large vocabulary size, large number of out of vocabulary tokens and different meaning for similar words.
Character based tokenization suffers from very large sequences and less meaningful individual tokens.

The pre-tokenization leaves us with a superset of all the tokens. The Dictionary training phase involves applying an algorithm to finalize the subset of tokens from this superset to be used for encoding.

Dont get it confused by machine learning tranining process. By train, this method is suppose to use a bunch of rules to produce an optimum dictionary. 
Using rules, the tokens are further split to form a compact vocabulary, at the same time reduce the chances of having unknown token ids. 

The most commonly used dictionary training approaches are

1. BPE - Byte Pair Encoding
2. WordPiece
3. SentencePiece
4. Unigram


```{admonition} Character Level Encoding
The two biggest challenge with word-level tokenization and are the size of the vocabulary and the number of unknown tokens added as a part of encoding.
The vocabulary size has to be very large to decrease the number of uknown token, however it does not guarantee greate reduction of unknown tokens. Words are based on characters, how about we tokenize the individual characters and use an encoding for each character?

    input_corpus_encoded = [ord(character) for character in text_corpus]
    print(input_corpus_encoded)
    assert len(text_corpus) == len(input_corpus_encoded)
    
    [76, 97, 114, 103, 101, 32, 108, 97, 110, 103, 117, 97, 103, 101, 32, 109, 111, 100, 101, 108, 115, 44, 32, 116, 104, 101, 32, 110, 101, 119, 32, 107, 105, 100, 32, 105, 110, 32, 116, 104, 101, 32, 98, 108, 111, 99, 107, 32, 105, 115, 32, 99, 114, 101, 97, 116, 105, 110, 103, 32, 119, 111, 110, 100, 101, 114, 115, 46, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 78, 117, 109, 101, 114, 111, 117, 115, 32, 97, 112, 112, 108, 105, 99, 97, 116, 105, 111, 110, 115, 32, 104, 97, 118, 101, 32, 115, 112, 97, 110, 110, 101, 100, 32, 105, 110, 32, 116, 104, 101, 32, 108, 97, 115, 116, 32, 116, 119, 111, 32, 121, 101, 97, 114, 115, 32, 108, 101, 118, 97, 114, 97, 103, 105, 110, 103, 32, 108, 108, 109, 115, 46]

    decoded_input = [chr(token_id) for token_id in input_corpus_encoded]
    print("".join(decoded_input))
    
    Large language models, the new kid in the block is creating wonders.                 Numerous applications have spanned in the last two years levaraging llms.

**Challenges with character level encoding**

The context of thw words are lost while doing character level encoding. They may be suitable for small toy llm's for unusable for building systems of any practical use.

```


#### Bye pair encoding

Byte pair encoding was first introduced for word segmentation in the paper Neural Machine Translation of Rare Words with Subword Units {cite}`sennrich2016neural`. It is a sub-word level method. The original algorithm is attribtued to Philip Gage. 1994. A New Algorithm for Data Com-pression. C Users J., 12(2):23–38, February. It is a data compression algorithm working iteratively. Say for example, we have the following string

aaaabdaaabac

Iteratively, let us now replace the most frequent pairs with another symbol not present in the string. For example, we replace the pair 'aa' with Z. The new
string will be ZabdZabac. 'Za' is the most frequently occuring pair now. Let us replace it with X. We continue this way till the string reaches the desired size.
Below is the python code to demonstrate this iteration.

In [45]:
def bpe_compression(input_str, desired_size=5):
    iterations = 0
    replace = ['Z','X','Y','L']
    replacements = []
    while len(input_str) > desired_size:
        iterations+=1
        pairs = Counter([i + j for i, j in zip(input_str, input_str[1:])])
        pair,freq = pairs.most_common(1)[0]
        if freq <=1 :
            break
        old_str = input_str
        input_str = input_str.replace(pair, replace[iterations - 1])
        replacements.append((replace[iterations -1], pair))
        print(f"Iteration {iterations} \n old {old_str} \n new {input_str} \n {pair} replaced by {replace[iterations-1]}")
    return input_str, replacements

input_str, replacements = bpe_compression('aaabdaaabac')
        

Iteration 1 
 old aaabdaaabac 
 new ZabdZabac 
 aa replaced by Z
Iteration 2 
 old ZabdZabac 
 new XbdXbac 
 Za replaced by X
Iteration 3 
 old XbdXbac 
 new YdYac 
 Xb replaced by Y


In order to reconstruct this, we store all the replacements in a stack.

In [46]:
print(replacements)

[('Z', 'aa'), ('X', 'Za'), ('Y', 'Xb')]


Now we can pop up the replacements from the stack and retrieve the original string.

In [48]:
while len(replacements) > 0:
    replace_str, _str = replacements.pop()
    print(replace_str, _str)
    input_str = input_str.replace(replace_str, _str)
    print(input_str)

BPE begins with the output from pre-tokenizer. For each token, a map of token with its constituent characters followed by a end of word symbol and its frequency in the input corpus are retrieved.

Let us see the example from {cite}`sennrich2016neural`.



In [71]:
vocab  = {'l o w </w>' : 5, 'l o w e r </w>' : 2, 'n e w e s t </w>':6, 'w i d e s t </w>':3}

So given a token 'l o w </w>', we get the list of subsequent character pairs and their frequency. In this case it will be

'l o', 'o w' and 'w </w>'. Since 'l o' occurs in 'l o w' and 'l o w e r',its frequency will be 7. 

In [72]:
from collections import defaultdict

def get_freq_pairs():
    pairs = defaultdict(int)
    for token,frequency in vocab.items():
        symbols = token.split()
        for i in range(len(symbols) -1):
            pair = (symbols[i],symbols[i+1])
            pairs[pair]+=frequency
    return pairs

pairs = get_freq_pairs()
pairs

defaultdict(int,
            {('l', 'o'): 7,
             ('o', 'w'): 7,
             ('w', '</w>'): 5,
             ('w', 'e'): 8,
             ('e', 'r'): 2,
             ('r', '</w>'): 2,
             ('n', 'e'): 6,
             ('e', 'w'): 6,
             ('e', 's'): 9,
             ('s', 't'): 9,
             ('t', '</w>'): 9,
             ('w', 'i'): 3,
             ('i', 'd'): 3,
             ('d', 'e'): 3})

In [73]:
best = max(pairs, key=pairs.get)
best = " ".join(best)

Now let us rebuild our vocabulary with this newly found frequencies. This is the merge operation. For that let us first get the most frequent
pair.

In [74]:
def merge(best, vocab_in):
    new_vocab = defaultdict(int)

    for token,freq in vocab_in.items():
        if best in token:
            print(best)
            best_concated = best.replace(" ","")
            token = token.replace(best, best_concated)
        new_vocab[token] = freq

    return new_vocab

vocab = merge(best, vocab)
print(vocab)

e s
e s
defaultdict(<class 'int'>, {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3})


In [81]:
vocab  = {'l o w </w>' : 5, 'l o w e r </w>' : 2, 'n e w e s t </w>':6, 'w i d e s t </w>':3}
rules = []
rule_number = 1
for i in range(5):
    pairs = get_freq_pairs()
    best = max(pairs, key=pairs.get)
    rules.append((rule_number," ".join(best), "".join(best)))
    rule_number+=1
    best = " ".join(best)
    vocab = merge(best, vocab)

print(vocab)
print(rules)

e s
e s
es t
es t
est </w>
est </w>
l o
l o
lo w
lo w
defaultdict(<class 'int'>, {'low </w>': 5, 'low e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3})
[(1, 'e s', 'es'), (2, 'es t', 'est'), (3, 'est </w>', 'est</w>'), (4, 'l o', 'lo'), (5, 'lo w', 'low')]


In [70]:
token_id = 0
final_vocab = {}
for token,freq in vocab.items():
    symbols = token.split()
    for symbol in symbols:
        if symbol not in final_vocab.keys():
            token_id+=1
            final_vocab[symbol] = token_id

print(final_vocab)

{'low': 1, '</w>': 2, 'e': 3, 'r': 4, 'n': 5, 'w': 6, 'est</w>': 7, 'i': 8, 'd': 9}


In [84]:
def encode_token(test):
    for rule in rules:
        r_no = rule[0]
        pattern = rule[1]
        replacement = rule[2]
        test = test.replace(pattern, replacement)
        
        print(f"Rule no {r_no} pattern {pattern} replacement {replacement} result {test}")
    
    encoded = [final_vocab[item] for item in test.split(" ")]
    print(encoded)

test = 'l o w e r </w>'
encode_token(test)

Rule no 1 pattern e s replacement es result l o w e r </w>
Rule no 2 pattern es t replacement est result l o w e r </w>
Rule no 3 pattern est </w> replacement est</w> result l o w e r </w>
Rule no 4 pattern l o replacement lo result lo w e r </w>
Rule no 5 pattern lo w replacement low result low e r </w>
[1, 3, 4, 2]


In [85]:
test = 'l o w e s t </w>'
encode_token(test)

Rule no 1 pattern e s replacement es result l o w es t </w>
Rule no 2 pattern es t replacement est result l o w est </w>
Rule no 3 pattern est </w> replacement est</w> result l o w est</w>
Rule no 4 pattern l o replacement lo result lo w est</w>
Rule no 5 pattern lo w replacement low result low est</w>
[1, 7]


In [ ]:
An implementation is available in https://github.com/openai/tiktoken open OpenAI.


### Post Processing

In the above example we used a special token **<UNKN>** to handle words which are not in the vocabulary. Some of the additional special tokens include

1. <BOS>, beginning of a sequence, a token to symbolize beginning of a text. This will help LLM understand where the text content begins.
2. <EOS>, end of sequence, a token to symbolize where the text begins.

LLMs are trained using multiple corpuses. These tokens helps them idenify when a token begins and when it ends
Let us add an additional corpus, add <BOS> and <EOS> tokens to the corpus and build the vocabulary.

### Toy token encoding pipeline

In [ ]:
from dataclasses import dataclass

@dataclass
class SpecialTokens:
    self.unknwn_token
    self.begin_token
    self.end_token


class ToyTextEncoder():
    """
    A simple example text encoder
    """
    def __init__(self, tokenizer, special_tokens) -> None:

        self.tokens = []
        self.vocabulary = {}
        self.reverse_vocabulary = {}
        self.tokenizer = tokenizer
        # assert special tokens
        self.special_tokens = special_tokens

    def __pretokenize(self, input_text: str):

        tokens = self.tokenizer(input_text)

        return tokens


    def __normalize(self, input_text: str):

        input_text = ftfy.fix_text(input_text)
        input_text = input_text.lower()

        return input_text


    def tokenize_text(self, input_text):

        assert len(input_text) > 0
        normal_text = self.__normalize(input_text)
        tokens = self.__pretokenize(normal_text)

        return tokens
        

    def __build_vocabulary(self):

        idx_ = 1
        self.vocabulary[self.special_tokens.unkwn_token] = idx_
        self.reverse_vocabulary[idx_] = self.special_tokens.unkwn_token
        idx_+=1
        self.vocabulary[self.special_tokens.begin_token] = idx_
        self.reverse_vocabulary[idx_] = self.special_tokens.begin_token
        idx+=1
        self.vocabulary[self.special_tokens.end_token]   = idx_
        self.reverse_vocabulary[idx_] = self.special_tokens.end_token
        idx+=1
        

        for idx, token in enumerate(self.tokens):
            self.vocabulary[token] =  idx + idx_
            self.reverse_vocabulary[idx + idx_] = token
    

    def train(self, input_corpus):

        if not isinstance(input_corpus, (list,tuple)):
            if isinstance(input_corpus, str):
                input_corpus = [input_corpus]

        for line in input_corpus:
            tokens = self.tokenize_text(line)
            if len(tokens) > 0:
                for token in tokens:
                    if token not in self.tokens:
                        self.tokens.append(token)


        
    def encode_text(self, input_text):

        encoding = []
        tokens = self.tokenize_text(input_text)
        if len(tokens) > 0:
            for token in tokens:
                encoding.append(self.vocabulary[token])

        return encoding
        
    def decode_text(self, input_tokenids):
        return [self.reverse_vocabulary[id] for id in input_tokenids]


In [139]:
encoder = ToyTextEncoder(tokenizer)
encoder.train(simple_books_sample)

Let us peek into the dictionary

In [140]:
for idx, (key,value) in enumerate(encoder.vocabulary.items()):
    print(f"{key} --> {value}")
    if idx == 5:
        break

more --> 0
jataka --> 1
tales --> 2
by --> 3
ellen --> 4
c --> 5


Now we can convieniently encode and decode any text

In [6]:
text = "tales by ellen king"
encoder.encode_text(text)

NameError: name 'encoder' is not defined

In [142]:
encoder.decode_text([2,3,4])

['tales', 'by', 'ellen']

As you can see in the above output, we have lost three tokens. Since the original words were not present in the dictionary, they were replaced by <UNKN> token.
During decoding we have lost the original three tokens.

::::{important}
:::{note}
While choosing the tokenizer a key requirment is that no information should be lost during encoding tokens to token-ids.  
:::
::::


## Training a dictionary

In [24]:
from datasets import load_dataset

In [25]:
dataset = load_dataset('../data/simplebooks/simplebooks-2-raw/')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [26]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 114696
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 13384
    })
    test: Dataset({
        features: ['text'],
        num_rows: 14830
    })
})

In [34]:
def get_train_corpus():
    return(
        dataset['train'][i:i+100]['text'] for i in range(0, len(dataset['train']),100)
    )

training_corpus = get_train_corpus()

from transformers import AutoTokenizer
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

sample = next(training_corpus)
tokens = old_tokenizer.tokenize(sample[0])
tokens

['More', 'ĠJ', 'ataka', 'ĠTales']

In [36]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 25000)

In [37]:
tokenizer.tokenize(sample[0])

['More', 'ĠJ', 'at', 'ak', 'a', 'ĠTales']

In [38]:
tokenizer.save_pretrained("../data/simplebooks-tokenizer")

('../data/simplebooks-tokenizer/tokenizer_config.json',
 '../data/simplebooks-tokenizer/special_tokens_map.json',
 '../data/simplebooks-tokenizer/vocab.json',
 '../data/simplebooks-tokenizer/merges.txt',
 '../data/simplebooks-tokenizer/added_tokens.json',
 '../data/simplebooks-tokenizer/tokenizer.json')

In [39]:
simplebooks_tokenizer = AutoTokenizer.from_pretrained("../data/simplebooks-tokenizer")
simplebooks_tokenizer.tokenize(sample[0])

['More', 'ĠJ', 'at', 'ak', 'a', 'ĠTales']

In [43]:
simplebooks_tokenizer.encode(sample[0])

[6628, 474, 275, 520, 65, 10578]

### Simple Books Pytorch Dataset

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

def get_train_corpus():
    return(
        dataset['train'][i:i+100]['text'] for i in range(0, len(dataset['train']),100)
    )

def get_validation_corpus():
    return(
        dataset['validation'][i:i+100]['text'] for i in range(0, len(dataset['validation']),100)
    )

training_corpus = get_train_corpus()
validation_corpus = get_validation_corpus()


class SimpleBooksDataSet(Dataset):
    """

    """
    def __init__(self, corpus_generator, max_length, stride):
        """

        """
        self.tokenizer  = AutoTokenizer.from_pretrained("../data/simplebooks-tokenizer")
        self.input_ids  = []
        self.target_ids = []
        self.token_ids  = []

        for sample in next(corpus_generator):
            """Returns 100 sentences"""
            for sentence in sample:
                self.token_ids.append(self.tokenizer(sample))

        for i in range(0, len(self.token_ids) - max_length,stride):
            input_chunk =  self.token_ids[i:i + max_length]
            target_chunk = self.token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.apppend(torch.tensor(target_chunk))


    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


simplebooks_train_ds = SimpleBooksDataSet(training_corpus, max_length=50, stride=5)            r
simplebooks_validation_ds = SimpleBooksDataSet(validation_corpus, max_length=50, stride=5)            r
            

        

train_dataloader = DataLoader()


        
        
        

## Word embedding


In [42]:
import torch
import torch.nn as nn




[6628, 474, 275, 520, 65, 10578]

## Position Embedding


* Absolute PE

Every input token at position i will be associated with a trainable embedding vector.

* Relative PE

Represents the distance between tokens.

"A women is nothing without her man"
"A man is nothing without her women"

These two sentences share the same words. They will hence share the same embeddings. For neural network both the sentences mean the same.
But we know they convey a different meaning. 



## Position Encoding

https://www.reddit.com/r/MachineLearning/comments/cttefo/d_positional_encoding_in_transformer/


 In attention, we basically take two word embeddings (x and y), pass one through a Query transformation matrix (Q) and the second through a Key transformation matrix (K), and compare how similar the resulting query and key vectors are by their dot product. So, basically, we want the dot product between Qx and Ky, which we write as:

(Qx)'(Ky) = x' (Q'Ky). So equivalently we just need to learn one joint Query-Key transformation (Q'K) that transform the secondary inputs y into a new space in which we can compare x.

By adding positional encodings e and f to x and y, respectively, we essentially change the dot product to

(Q(x+e))' (K(y+f)) = (Qx+Qe)' (Ky+Kf) = (Qx)' Ky + (Qx)' Kf + (Qe)' Ky + (Qe)' Kf = x' (Q'Ky) + x' (Q'Kf) + e' (Q'Ky) + e' (Q'K f), where in addition to the original x' (Q'Ky) term, which asks the question "how much attention should we pay to word x given word y", we also have x' (Q'Kf) + e' (Q'Ky) + e' (Q'K f), which ask the additional questions, "how much attention should we pay to word x given the position f of word y", "how much attention should we pay to y given the position e of word x", and "how much attention should we pay to the position e of word x given the position f of word y".

Essentially, the learned transformation matrix Q'K with positional encodings has to do all four of these tasks simultaneously. This is the part that may appear inefficient, since intuitively, there should be a trade-off in the ability of Q'K to do four tasks simultaneously and well.

 HOWEVER, MY GUESS is that there isn't actually a trade-off when we force Q'K to do all four of these tasks, because of some approximate orthogonality condition that is satisfied of in high dimensions. The intuition for this is that randomly chosen vectors in high dimensions are almost always approximately orthogonal. There's no reason to think that the word vectors and position encoding vectors are related in any way. If the word embeddings form a smaller dimensional subspace and the positional encodings form another smaller dimensional subspace, then perhaps the two subspaces themselves are approximately orthogonal, so presumably these subspaces can be transformed approx. independently through the same learned Q'K transformation (since they basically exist on different axes in high dimensional space). I don't know if this is true, but it seems intuitively possible.

If true, this would explain why adding positional encodings, instead of concatenation, is essentially fine. Concatenation would ensure that the positional dimensions are orthogonal to the word dimensions, but my guess is that, because these embedding spaces are so high dimensional, you can get approximate orthogonality for free even when adding, without the costs of concatenation (many more parameters to learn). Adding layers would only help with this, by allowing for nonlinearities.

We also ultimately want e and f to behave in some nice ways, so that there's some kind of "closeness" in the vector representation with respect to small changes in positions. The sin and cos representation is nice since nearby positions have high similarity in their positional encodings, which may make it easier to learn transformations that "preserve" this desired closeness.

(Maybe I'm wrong, and the approximate orthogonality arises from stacking multiple layers or non-linearities in the fully-connected parts of the transformer).

tl;dr: It is intuitively possible that, in high dimensions, the word vectors form a smaller dimensional subspace within the full embedding space, and the positional vectors form a different smaller dimensional subspace approximately orthogonal to the one spanned by word vectors. Thus despite vector addition, the two subspaces can be manipulated essentially independently of each other by some single learned transformation. Thus, concatenation doesn't add much, but greatly increases cost in terms of parameters to learn. 



In [136]:
import torch

n = 100
d_model = 4
context_window = 5

position_embedding = torch.zeros(context_window, d_model)

"""
Embedding[i, 2k]   = sin(position / n ^ (2k/d_model))
Embedding[i, 2k+1] = cos(position / n ^ (2k/d_model))

i = position index in the sequence
k = dimension index within the embedding vector
"""

d_model % 2

0

In [137]:


for i in range(context_window):
    for k in range(0, d_model//2) :
        # Even positions in embedding vector
        even_idx = 2 * k
        denominator = torch.tensor(n ** (even_idx / d_model))
        position_embedding[i, even_idx] = torch.sin(i/denominator)
        # Odd positions in embedding vector
        odd_idx = 2*k + 1
        position_embedding[i, odd_idx] = torch.cos(i/denominator)

In [138]:
position_embedding

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0998,  0.9950],
        [ 0.9093, -0.4161,  0.1987,  0.9801],
        [ 0.1411, -0.9900,  0.2955,  0.9553],
        [-0.7568, -0.6536,  0.3894,  0.9211]])

Position embedding should be monotonic. Teh closer you are, the more your influence on the current token. But sinusoid embeddings totally
violate that.

ALIBI based encoding is monotonic and seems to do better than sinusoidal encoding.

## Conclustion

## Further Reading